In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.storage import LocalFileStore
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import UnstructuredFileLoader
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough

In [119]:
llm = ChatOpenAI(temperature=0.2)

memory = ConversationBufferMemory(
    return_message=True,
)

cache_dir = LocalFileStore("./.cache/")

In [4]:
loader = UnstructuredFileLoader("./files/document.txt")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

docs = loader.load_and_split(text_splitter=splitter)

In [8]:
len(docs)
docs[0]

Document(page_content='Chapter 3\n\'There are three stages in your reintegration,\' said O\'Brien. \'There is learning, there is understanding, and there is acceptance. It is time for you to enter upon the second stage.\'\nAs always, Winston was lying flat on his back. But of late his bonds were looser. They still held him to the bed, but he could move his knees a little and could turn his head from side to side and raise his arms from the elbow. The dial, also, had grown to be less of a terror. He could evade its pangs if he was quick-witted enough: it was chiefly when he showed stupidity that O\'Brien pulled the lever. Sometimes they got through a whole session without use of the dial. He could not remember how many sessions there had been. The whole process seemed to stretch out over a long, indefinite time--weeks, possibly--and the intervals between the sessions might sometimes have been days, sometimes only an hour or two.\n\'As you lie there,\' said O\'Brien, \'you have often won

In [9]:
embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

In [146]:
retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]

In [147]:
chain = (
    {
        "context": retriver,
        "question": RunnablePassthrough(),
    }
    | RunnablePassthrough.assign(history=load_memory)
    | prompt
    | llm
)

In [148]:
memory.clear()

memory = ConversationBufferMemory(return_messages=True)


def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context({"input": question}, {"output": result.content})
    print(result)

In [149]:
invoke_chain("Aaronson 은 유죄인가요?")

content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 제가 가지고 있지 않습니다.'


In [150]:
invoke_chain("그가 테이블에 어떤 메시지를 썼나요?")

content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER'


In [151]:
invoke_chain("Julia 는 누구인가요?")

content='Julia는 주인공이 사랑하는 여성 캐릭터입니다.'


In [152]:
memory.load_memory_variables({})["history"]

[HumanMessage(content='Aaronson 은 유죄인가요?'),
 AIMessage(content='제가 알기로는 Aaronson이 유죄인지 아닌지에 대한 정보는 제가 가지고 있지 않습니다.'),
 HumanMessage(content='그가 테이블에 어떤 메시지를 썼나요?'),
 AIMessage(content='그가 쓴 메시지는 다음과 같습니다:\nFREEDOM IS SLAVERY\nTWO AND TWO MAKE FIVE\nGOD IS POWER'),
 HumanMessage(content='Julia 는 누구인가요?'),
 AIMessage(content='Julia는 주인공이 사랑하는 여성 캐릭터입니다.')]